In [1]:
import torch
from torch.nn import LSTM
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.func import functional_call
from torch.nn.functional import normalize

import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

from dataset import TSDataset
from models import LSTMModel, AttentionModel
from train_utils import train_and_test
from neural_memory import NeuralMemory
from titans import MACTitanLayer, MACTitan

from tqdm import tqdm
from copy import deepcopy

torch.set_grad_enabled(True)

# ===========================================================================================

train_data = TSDataset('sinwave', 'train', False)
valid_data = TSDataset('sinwave', 'valid', False)
test_data = TSDataset('sinwave', 'test', False)

_train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
_valid_loader = DataLoader(valid_data, batch_size=4, shuffle=True)
_test_loader = DataLoader(test_data, batch_size=4, shuffle=True)

In [23]:
model = LSTMModel(1, 16, 16)

optimizer = optim.Adam(model.parameters())

train_and_test(model, optimizer, _train_loader, _valid_loader, _test_loader, 50)

Training:


100%|██████████| 50/50 [00:24<00:00,  2.07it/s]


 Testing the best model:
Test MSE: 0.015290285992835249



LSTMModel(
  (emb_layer): Linear(in_features=1, out_features=16, bias=True)
  (relu): ReLU()
  (lstm): LSTM(16, 16, batch_first=True)
  (final_layer): Linear(in_features=16, out_features=1, bias=True)
)

In [24]:
model = AttentionModel(1, 16, 16)

optimizer = optim.Adam(model.parameters())

train_and_test(model, optimizer, _train_loader, _valid_loader, _test_loader, 50)

Training:


100%|██████████| 50/50 [06:38<00:00,  7.97s/it]



 Testing the best model:
Test MSE: 0.03654162958264351



AttentionModel(
  (emb_layer): Linear(in_features=1, out_features=16, bias=True)
  (att_layers): ModuleList(
    (0-1): 2 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
      )
      (linear1): Linear(in_features=16, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=16, bias=True)
      (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (activation): SiLU()
    )
  )
  (final_layer): Linear(in_features=256, out_features=1, bias=True)
)

In [25]:
model = MACTitan(
    input_dim = 1,
    hidden_dim = 16,
    output_dim = 1,
    context_window = 16,
    pm_len = 4,
    n_layers = 2,
    n_layers_nmm = 2,
    eta=0.9,
    theta=0.01
).cuda()

optimizer = optim.Adam(model.outer_params)

train_and_test(model, optimizer, _train_loader, _valid_loader, _test_loader, 25)

Training:


  4%|▍         | 1/25 [01:04<25:40, 64.17s/it]

Validation loss in Epoch 0: 0.0464873415018831


  8%|▊         | 2/25 [02:09<24:49, 64.75s/it]

Validation loss in Epoch 1: 0.0319312254765204


...

Validation loss in Epoch 23: 0.019545881130865642


100%|██████████| 25/25 [27:08<00:00, 65.12s/it]

Validation loss in Epoch 24: 0.018531270511448383

 Testing the best model:


Test MSE: 0.01826440165085452



MACTitan(
  (emb_layer): Linear(in_features=1, out_features=16, bias=True)
  (layers): ModuleList(
    (0-1): 2 x MACTitanLayer(
      (att_layer): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
        )
        (linear1): Linear(in_features=16, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=16, bias=True)
        (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
        (activation): SiLU()
      )
      (Q): Linear(in_features=16, out_features=16, bias=True)
      (nm_module): NeuralMemory(
        (layers): ModuleList(
          (0): Sequential(
            (0): Linear(in_features=16, out_features=32,